In [1]:
import sys
import time
import torch
import numpy as np
from sklearn.metrics import f1_score

sys.path.append('../')
from fslnet.fslnet import FSLNet

In [2]:
# Automatically select GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)

Device: cpu


In [ ]:
fslnet = FSLNet.from_pretrained(device=device)

--> Loading FSLNet weights from 'C:\Users\janag\AppData\Roaming\Python\Python313\site-packages\fslnet\checkpoints\fslnet.pth' onto cpu ...
--> FSLNet loaded and set to eval().


In [5]:
ref_path = '../data/covid_0.1_E1_ref.npy'
que_path = '../data/covid_0.1_E1_que.npy'
C_positions_path = '../data/covid_0.1_E1_C_positions.npy'

ref = torch.tensor(np.load(ref_path), dtype=torch.float32)
que = torch.tensor(np.load(que_path), dtype=torch.float32)
C_positions = torch.tensor(np.load(C_positions_path))

In [6]:
with torch.no_grad():
    start_time = time.time()
    soft_predictions, _ = fslnet(ref, que)        # Corruption probabilities
    hard_predictions = (soft_predictions > 0.5)  # Boolean mask — True = shifted
    end_time = time.time()

runtime = end_time - start_time

# Create ground truth tensor
target = torch.zeros(1, que.shape[1]).to(device)
target[0, C_positions] = 1

# Compute F1 Score
f1 = f1_score(target.squeeze(), hard_predictions.squeeze(), zero_division=1)

print("F1 Score:", f1)
print("Runtime (seconds):", runtime)

F1 Score: 1.0
Runtime (seconds): 0.42274045944213867
